## Trajectory tutorial  - 2022 0127 CJH
* We need to generate a list of characteristics for the robot to track, bundled into a trajectory.  
* Trajectories give you:
  * A list of states with t, velocity, acceleration, pose=Pose2d(Translation2d(x, y), Rotation2d(rad)), curvature)
  * A sample() method to get the state at any time
  * A totalTime member
  * a transformBy so we can change the starting point
  
* We need input to the trajectory - the points it must follow
  * pathweaver **paths** (not to be confused with python *Paths* or operating systems directories) do this
  * we can do it with lists of points as well


---

#### Auto generating trajecotories to follow using wpilib


In [1]:
import glob
from pathlib import Path

from wpimath.geometry import Pose2d, Rotation2d, Translation2d, Transform2d, Twist2d
import wpimath.trajectory
from wpimath.kinematics import DifferentialDriveKinematics
from wpimath.trajectory.constraint import DifferentialDriveVoltageConstraint, CentripetalAccelerationConstraint
import wpilib.controller

#### unfortunately, trajectories need to know a lot about the robot

In [7]:
# All the characterization constants and drive attributes we need to generate a trajectory
ks_volts = 0.446
kv_volt_seconds_per_meter = 1.55
ka_volt_seconds_squared_per_meter = 0.40
k_track_width_meters = 24 * 0.0254
k_max_voltage = 6
k_max_centripetal_acceleration_meters_per_second_squared = 2.75
k_max_acceleration_meters_per_second_squared = 2.75
k_drive_kinematics = DifferentialDriveKinematics(k_track_width_meters)
k_feed_forward = wpilib.controller.SimpleMotorFeedforwardMeters(ks_volts, kv_volt_seconds_per_meter, ka_volt_seconds_squared_per_meter)
k_autonomous_voltage_constraint = DifferentialDriveVoltageConstraint(k_feed_forward, k_drive_kinematics, k_max_voltage)

#### the most basic trajectory - connect two poses

In [64]:
start_pose = Pose2d(Translation2d(x=0, y=0), Rotation2d(0.000000))
end_pose = Pose2d(Translation2d(x=1, y=0), Rotation2d(0))
velocity = 1
config = wpimath.trajectory.TrajectoryConfig(velocity, k_max_acceleration_meters_per_second_squared)
traj = wpimath.trajectory.TrajectoryGenerator.generateTrajectory(waypoints=[start_pose, end_pose], config=config)
traj.states()

[Trajectory.State(t=0.000000, velocity=0.000000, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.000000, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.232858, velocity=0.640359, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.074556, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.326735, velocity=0.898522, acceleration=1.407149, pose=Pose2d(Translation2d(x=0.146790, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.398851, velocity=1.000000, acceleration=0.000000, pose=Pose2d(Translation2d(x=0.215246, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.462902, velocity=1.000000, acceleration=0.000000, pose=Pose2d(Translation2d(x=0.279297, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.578563, velocity=1.000000, acceleration=0.000000, pose=Pose2d(Translation2d(x=0.394958, y=0.000000), Rotation2d(0.000000)), curvature=0.000000)

#### the full on, on-the fly version of generating trajectories on the robot

In [2]:
pwd

'C:\\Cory\\CJH Shared\\Python\\FRC\\FRC_training\\notebooks\\controls'

In [3]:
path_location = r'..\..\..\2022\robot\pathweaver\paths\\'
# find all of the pathweaver paths available to the robot - use to populate a drop-down sendable chooser in the UI
def get_pathweaver_paths():  # use this to fill the drop down for file selection
    # instead of passing robot simulation, check for directory exists on the robot
    path_files = glob.glob(path_location + '*', recursive=True)
    #print(f'** Pathweaver files: {path_files} **')
    path_names = [Path(file).name for file in path_files]
    return path_names

In [4]:
paths = get_pathweaver_paths()
paths

['2_circle',
 '2_meters',
 'barrel_pw0',
 'barrel_pw1',
 'bounce_pw0',
 'bounce_pw1',
 'bounce_pw2',
 'bounce_pw3',
 'bounce_pw4',
 'calibrate',
 'calibrate_rev',
 'slalom_pw0_square',
 'slalom_pw1_mixed',
 'slalom_pw4_smooth']

In [5]:
Path(path_location) / paths[0]

WindowsPath('../../../2022/robot/pathweaver/paths/2_circle')

In [6]:
def generate_trajectory(path_name:str, velocity=2, display=True, save=True) -> wpimath.trajectory:
    """
    Generate a wpilib trajectory from a pathweaver path.  Accepts regular and reversed paths.

    :param path_name: name of pathweaver file to be imported
    :param velocity: Maximum robot velocity for the generated trajectory
    :param save: Option to save generated trajectory to disk as 'test.json'
    :param display: Option to print to console
    :return: generated trajectory
    """

    pathweaver_y_offfset = 4.572
    p = Path(path_location + path_name)
    if p.is_file():
        lines = []
        with open(p, "r") as f:
            for line in f:
                currentline = line.split(",")
                lines.append(currentline)
        cvector_list = [wpimath.spline.Spline5.ControlVector((float(row[0]), float(row[2]), 0), (float(row[1]) + pathweaver_y_offfset, float(row[3]), 0))
                        for ix, row in enumerate(lines[1:])]

        config = wpimath.trajectory.TrajectoryConfig(velocity, k_max_acceleration_meters_per_second_squared)
        config.setKinematics(k_drive_kinematics)
        config.addConstraint(k_autonomous_voltage_constraint)
        config.addConstraint(CentripetalAccelerationConstraint(k_max_centripetal_acceleration_meters_per_second_squared))
        # check to see if the path is to be reversed it is marked in the th column of the pathweaver file
        reverse_array = [row[5] for row in lines[1:]]
        if any(entry == 'true' for entry in reverse_array):
            config.setReversed(True)
        pw_trajectory = wpimath.trajectory.TrajectoryGenerator.generateTrajectory(cvector_list, config)
        if save:
            wpimath.trajectory.TrajectoryUtil.toPathweaverJson(pw_trajectory, 'pathweaver\\test.json')
    else:
        pw_trajectory = None  # do something else? generate an empty trajectory?
        print(f'Trajectory: {p} not found', flush=True)
    if display:
        print(pw_trajectory)
    return pw_trajectory

In [8]:
traj = generate_trajectory(path_name=paths[1], velocity=2, display=True, save=False)

In [9]:
traj_attribs = [attrib for attrib in dir(traj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(traj_attribs) ]

State, initialPose, relativeTo, sample, states, totalTime, transformBy, 

In [16]:
traj.initialPose()

Pose2d(Translation2d(x=1.000000, y=0.572000), Rotation2d(0.000000))

In [10]:
traj.sample(t=1.5)

Trajectory.State(t=1.500000, velocity=0.625782, acceleration=-2.750000, pose=Pose2d(Translation2d(x=2.928799, y=0.572000), Rotation2d(0.000000)), curvature=0.000000)

In [11]:
# show all of the states of the trajectory
traj.states()

[Trajectory.State(t=0.000000, velocity=0.000000, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.000000, y=0.572000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.254496, velocity=0.699864, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.089056, y=0.572000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.356006, velocity=0.979016, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.174268, y=0.572000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.463589, velocity=1.274870, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.295508, y=0.572000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.518443, velocity=1.425718, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.369577, y=0.572000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.573322, velocity=1.576637, acceleration=2.750000, pose=Pose2d(Translation2d(x=1.451961, y=0.572000), Rotation2d(0.000000)), curvature=0.000000)

In [43]:
traj.states()[-1].pose

Pose2d(Translation2d(x=3.000000, y=0.572000), Rotation2d(0.000000))

#### let's move the trajectory

In [35]:
traj.transformBy?

Docstring:
transformBy(self: wpimath.trajectory._trajectory.Trajectory, transform: wpimath.geometry._geometry.Transform2d) -> wpimath.trajectory._trajectory.Trajectory

Transforms all poses in the trajectory by the given transform. This is
useful for converting a robot-relative trajectory into a field-relative
trajectory. This works with respect to the first pose in the trajectory.

:param transform: The transform to transform the trajectory by.

:returns: The transformed trajectory.
Type:      method


In [36]:
new_start_pose = Pose2d(Translation2d(0, 0), Rotation2d(0)) 

In [37]:
# can just do it directly
transform = Transform2d(Translation2d(x=-1, y=-0.572), Rotation2d(0))
transform

Transform2d(Translation2d(x=-1.000000, y=-0.572000), Rotation2d(0.000000))

In [38]:
# or can use the initial and final poses
Transform2d(traj.initialPose(), new_start_pose)

Transform2d(Translation2d(x=-1.000000, y=-0.572000), Rotation2d(0.000000))

In [40]:
new_traj = traj.transformBy(transform)

In [41]:
new_traj.states()  # now we start at zero

[Trajectory.State(t=0.000000, velocity=0.000000, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.000000, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.254496, velocity=0.699864, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.089056, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.356006, velocity=0.979016, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.174268, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.463589, velocity=1.274870, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.295508, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.518443, velocity=1.425718, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.369577, y=0.000000), Rotation2d(0.000000)), curvature=0.000000),
 Trajectory.State(t=0.573322, velocity=1.576637, acceleration=2.750000, pose=Pose2d(Translation2d(x=0.451961, y=0.000000), Rotation2d(0.000000)), curvature=0.000000)